In [1]:
import pandas as pd
import requests

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.intraday_data = {}
    
    def fetch_intraday_data(self, script):
        # Send request to Alpha Vantage API to fetch intraday stock data
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=60min&apikey={self.api_key}"
        response = requests.get(url)
        self.intraday_data[script] = response.json()
    
    def convert_intraday_data(self, script):
        data = self.intraday_data[script]["Time Series (60min)"]
        # Convert raw data to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index', dtype=float)
        df.reset_index(inplace=True)
        df.rename(columns={"index": "timestamp"}, inplace=True)
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.sort_values("timestamp",ascending=False, inplace=True)
        df = df[["timestamp", "1. open", "2. high", "3. low", "4. close", "5. volume"]]
        df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"}, inplace=True)
        df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].astype(float)
        self.intraday_data[script] = df
    
    def __getitem__(self, script):
         return self.intraday_data[script]
    
    def __setitem__(self, script, df):
        self.intraday_data[script] = df
    
    def __contains__(self, script):
        return script in self.intraday_data

In [2]:
script_data =ScriptData("IO1XAWEP2CEP28OE")
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

print(script_data['GOOGL'])

             timestamp     open   high    low  close     volume
0  2023-02-24 20:00:00  89.1600  89.23  89.13  89.13    20290.0
1  2023-02-24 19:00:00  89.1700  89.25  89.15  89.22     6531.0
2  2023-02-24 18:00:00  89.2800  89.30  89.13  89.24    27477.0
3  2023-02-24 17:00:00  89.1300  89.43  89.13  89.30   733696.0
4  2023-02-24 16:00:00  89.2100  89.37  88.84  89.16  6888953.0
..                 ...      ...    ...    ...    ...        ...
95 2023-02-16 05:00:00  97.1000  97.23  96.95  97.02    33881.0
96 2023-02-15 20:00:00  96.9300  97.12  96.80  97.10    84085.0
97 2023-02-15 19:00:00  96.9000  97.01  96.89  96.94    68819.0
98 2023-02-15 18:00:00  96.9406  96.96  96.86  96.90    61057.0
99 2023-02-15 17:00:00  96.9400  97.08  96.85  96.95   987263.0

[100 rows x 6 columns]


In [3]:
print('GOOGL' in script_data)

True


In [4]:
def indicator1(df, timeperiod):
    # Compute moving average of 'close' column using specified timeperiod
    moving_average = df["close"].rolling(window=timeperiod).mean()
    # Return pandas DataFrame with 'timestamp' and 'indicator' columns
    result_df = pd.DataFrame({"timestamp": df["timestamp"], "indicator": moving_average})
    return result_df

In [5]:
print(indicator1(script_data['GOOGL'],timeperiod=5))

             timestamp  indicator
0  2023-02-24 20:00:00        NaN
1  2023-02-24 19:00:00        NaN
2  2023-02-24 18:00:00        NaN
3  2023-02-24 17:00:00        NaN
4  2023-02-24 16:00:00     89.210
..                 ...        ...
95 2023-02-16 05:00:00     96.632
96 2023-02-15 20:00:00     96.952
97 2023-02-15 19:00:00     97.050
98 2023-02-15 18:00:00     97.010
99 2023-02-15 17:00:00     96.982

[100 rows x 2 columns]


In [6]:
class Strategy:
    def __init__(self, script, api_key,timeperiod):
        self.script = script
        self.script_data = ScriptData(api_key)
        self.timeperiod=timeperiod

    def get_script_data(self):
        self.script_data.fetch_intraday_data(self.script)
        self.script_data.convert_intraday_data(self.script)
        self.df = self.script_data[self.script]
        self.indicator_data = indicator1(self.df, self.timeperiod)
        
    def get_signals(self):
        # create an empty list to store the signals
        signals=[]
        for i in range(1, len(self.df)):
            if self.indicator_data["indicator"].iloc[i] > self.df["close"].iloc[i] and self.indicator_data["indicator"].iloc[i-1] <= self.df["close"].iloc[i-1]:
                signals.append({"timestamp": self.df["timestamp"].iloc[i], "signal": "BUY"})
            elif self.indicator_data["indicator"].iloc[i] < self.df["close"].iloc[i] and self.indicator_data["indicator"].iloc[i-1] >= self.df["close"].iloc[i-1]:
                signals.append({"timestamp": self.df["timestamp"].iloc[i], "signal": "SELL"})
            else:
                signals.append({"timestamp": self.df["timestamp"].iloc[i], "signal": "NO_SIGNAL"})
        # convert the signals list to a pandas DataFrame and store it in the self.signals attribute
        self.signals = pd.DataFrame(signals)
        new_signals = self.signals[(self.signals['signal'] == 'BUY') | (self.signals['signal'] == 'SELL')].reset_index(drop=True)
        return new_signals

In [7]:
strategy = Strategy('NVDA','IO1XAWEP2CEP28OE',5)
strategy.get_script_data()
output=strategy.get_signals()
print(output)

             timestamp signal
0  2023-02-24 15:00:00   SELL
1  2023-02-24 14:00:00    BUY
2  2023-02-24 12:00:00   SELL
3  2023-02-24 11:00:00    BUY
4  2023-02-24 09:00:00   SELL
5  2023-02-24 05:00:00    BUY
6  2023-02-23 20:00:00   SELL
7  2023-02-23 13:00:00    BUY
8  2023-02-23 11:00:00   SELL
9  2023-02-23 09:00:00    BUY
10 2023-02-22 20:00:00   SELL
11 2023-02-22 17:00:00    BUY
12 2023-02-22 10:00:00   SELL
13 2023-02-22 07:00:00    BUY
14 2023-02-21 19:00:00   SELL
15 2023-02-21 09:00:00    BUY
16 2023-02-21 08:00:00   SELL
17 2023-02-21 06:00:00    BUY
18 2023-02-17 20:00:00   SELL
19 2023-02-17 16:00:00    BUY
20 2023-02-17 15:00:00   SELL
21 2023-02-17 14:00:00    BUY
22 2023-02-17 10:00:00   SELL
23 2023-02-17 06:00:00    BUY
24 2023-02-17 05:00:00   SELL
25 2023-02-16 12:00:00    BUY
26 2023-02-16 08:00:00   SELL
